In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from collections import defaultdict
from scipy import stats

In [2]:
transactions = pd.read_csv("./data/dh_transactions.csv")
products = pd.read_csv("./data/dh_product_lookup.csv")

In [3]:
data = transactions.join(products.set_index('upc'),on = "upc")

since the next section we will do various probability calculating so removing an underperformed products/brands before hand will simplify the process a lot. In this case, We will remove products/brands that sold under 100 times across 2 years.

In [49]:
brands = data["brand"].value_counts()
products = data["product_description"].value_counts()

cut_brands = brands[brands <= 100].index
cut_products = products[products <= 100].index

data = data[~data["brand"].isin(cut_brands)]
data = data[~data["product_description"].isin(cut_products)]

In [50]:
baskets = data.groupby(['basket'])

# Calculate probability that customers will buy only that product/brand

In [51]:
baskets_with_single_product = []
for basket_name ,basket in baskets:
    if len(basket) == 1:
        baskets_with_single_product.append(basket_name)

/var/folders/lc/b2zpsxhx5wj0p33dk9d2hplh0000gn/T/ipykernel_4350/4008163490.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for basket_name ,basket in baskets:


In [52]:
single_product_data = data[data["basket"].isin(baskets_with_single_product)]

In [66]:
prob_single_commodities = single_product_data["commodity"].value_counts()/data["commodity"].value_counts()

In [64]:
prob_single_products = single_product_data["product_description"].value_counts()/data["product_description"].value_counts()

In [60]:
prob_single_brands = single_product_data["brand"].value_counts()/data["brand"].value_counts()

In [144]:
prob_single_brands.sort_values()

Hunt's             0.272681
Classico           0.289781
Ragu               0.289907
Bertolli           0.299365
Prego              0.309537
                     ...   
Grandma Molases    0.737790
Brier Rabbit       0.746555
Sobrab Bkstrp      0.749851
HSE                0.771739
La Moderna         0.853448
Name: brand, Length: 70, dtype: float64

In [154]:
prob_single_commodities.sort_values()

pasta sauce      0.294000
pasta            0.401447
pancake mixes    0.432527
syrups           0.608599
Name: commodity, dtype: float64

In [149]:
data[data["brand"] == "La Moderna"]["commodity"].unique()

array(['pasta'], dtype=object)

In [151]:
data[data["brand"] == "HSE"]["commodity"].unique()

array(['syrups'], dtype=object)

In [153]:
data[data["brand"] == "Sobrab Bkstrp"]["commodity"].unique()

array(['syrups'], dtype=object)

# Calculate Probability that customers will buy 2 given products/brands together

In [70]:
multi_product_data = data[~data["basket"].isin(baskets_with_single_product)]

In [73]:
multi_product_baskets = multi_product_data.groupby(['basket'])

In [129]:
commodity_collections = {}
check = 200000
s=time.time()
for basket_id ,basket in baskets:
    collection = defaultdict(int)
    
    # There are a case where the unit isn't 1 so we need to take that into account
    for _id,row in basket.iterrows():
        collection[row["commodity"]] += row["units"]
    commodity_collections[basket_id] = collection
    
    if basket_id > check:
        e = time.time()
        print(f"basket : {basket_id}, time: {e-s}")
        check += 200000


/var/folders/lc/b2zpsxhx5wj0p33dk9d2hplh0000gn/T/ipykernel_4350/4234669301.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for basket_id ,basket in baskets:


basket : 200001, time: 28.33632493019104
basket : 400001, time: 53.48141288757324
basket : 600001, time: 80.970449924469
basket : 800001, time: 109.08711910247803
basket : 1000001, time: 146.72439408302307
basket : 1200001, time: 180.51022219657898
basket : 1400001, time: 211.31916117668152
basket : 1600001, time: 242.3439440727234
basket : 1800001, time: 273.43626594543457
basket : 2000001, time: 304.72812509536743
basket : 2200001, time: 335.0639500617981
basket : 2400001, time: 365.55337405204773
basket : 2600001, time: 396.8131091594696
basket : 2800001, time: 428.25536584854126
basket : 3000001, time: 464.4826591014862
basket : 3200001, time: 496.30604815483093


In [130]:
brand_collections = {}
check = 200000
s=time.time()
for basket_id ,basket in baskets:
    collection = defaultdict(int)
    
    # There are a case where the unit isn't 1 so we need to take that into account
    for _id,row in basket.iterrows():
        collection[row["brand"]] += row["units"]
    brand_collections[basket_id] = collection
    
    if basket_id > check:
        e = time.time()
        print(f"basket : {basket_id}, time: {e-s}")
        check += 200000

/var/folders/lc/b2zpsxhx5wj0p33dk9d2hplh0000gn/T/ipykernel_4350/1779566045.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for basket_id ,basket in baskets:


basket : 200001, time: 34.547085762023926
basket : 400001, time: 65.5997519493103
basket : 600001, time: 96.57375597953796
basket : 800001, time: 127.02918291091919
basket : 1000001, time: 157.65265274047852
basket : 1200001, time: 187.9719638824463
basket : 1400001, time: 218.88635873794556
basket : 1600001, time: 250.01762986183167
basket : 1800001, time: 280.85636591911316
basket : 2000001, time: 311.5839099884033
basket : 2200001, time: 347.5334618091583
basket : 2400001, time: 381.2188286781311
basket : 2600001, time: 414.78519773483276
basket : 2800001, time: 446.57677698135376
basket : 3000001, time: 477.18012499809265
basket : 3200001, time: 508.55888271331787


In [131]:
product_collections = {}
check = 200000
s=time.time()
for basket_id ,basket in baskets:
    collection = defaultdict(int)
    
    # There are a case where the unit isn't 1 so we need to take that into account
    for _id,row in basket.iterrows():
        collection[row["product_description"]] += row["units"]
    product_collections[basket_id] = collection
    
    if basket_id > check:
        e = time.time()
        print(f"basket : {basket_id}, time: {e-s}")
        check += 200000

/var/folders/lc/b2zpsxhx5wj0p33dk9d2hplh0000gn/T/ipykernel_4350/3552992282.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for basket_id ,basket in baskets:


basket : 200001, time: 37.05305194854736
basket : 400001, time: 71.0735821723938
basket : 600001, time: 104.74058794975281
basket : 800001, time: 136.4825210571289
basket : 1000001, time: 166.55057501792908
basket : 1200001, time: 196.23241806030273
basket : 1400001, time: 225.62112402915955
basket : 1600001, time: 255.24258518218994
basket : 1800001, time: 291.74751687049866
basket : 2000001, time: 322.10955286026
basket : 2200001, time: 352.9223539829254
basket : 2400001, time: 383.54249596595764
basket : 2600001, time: 413.88081884384155
basket : 2800001, time: 445.9849741458893
basket : 3000001, time: 476.2845780849457
basket : 3200001, time: 507.40276193618774


In [124]:
# Calculate P(A and B)
def cal_prob(product_group, collections):
    c = 0
    for basket_id in collections:
        collection = collections[basket_id]
        tracker = 0
        for key in product_group.keys():
            if product_group[key] <= collection.get(key,0):
                tracker += 1
            
        if tracker == len(product_group):
            c += 1
    return c / len(collections)

# Calculate P(A given B)
def bayes( A, B, feature):
    if feature == "brand":
        collections = brand_collections
    elif feature == "commodity":
        collections = commodity_collections
    elif feature == "product":
        collections = product_collections
    
    group = {}
    group[A] = group.get(A,0) + 1
    group[B] = group.get(B,0) + 1
    
    p_a_and_b = cal_prob(group, collections)
    
    # Calculate P(B)
    c = 0
    for basket_id in collections:
        collection = collections[basket_id]
        if B in collection:
            c += 1
    p_b = c / len(collections)
    
    return p_a_and_b/p_b

In [155]:
bayes("pancake mixes", "syrups", "commodity")

0.16342623548720445

## Conclusion

With this calculation, we can get a relation between each commodity/product/brand, which we can use this to design more sophisticated promotion instead of simple discount coupon.

For example,
1. There are 41.78% chance that a customer who buy pasta will also buy pasta sauce.<br>
    -This information can support us if we want to launch a campaign like "You will get 10% discount when buying pasta and pasta sauce together"<br>
2. There are 40.79% chance that a customer who buy pancake mixes will also buy syrups.<br>
    -With this information, we can tell our cashiers that if a customer come to their station with pancake mixes but no syrups, they can offer syrups at special price for that customer.<br>
3. If customers buy product from Private Label brand, there are 14.49% that they will additionally buy product from Ragu Brand or 6.73% that they will add product from Prego to their basket.<br>
    -in case we want to pick a brand to pair with Private Label in our new campaign, we can use this to calculate and see that Ragu is more suitable brand for this campaign than Prego.<br>
    
This can also combine with the previous section analysis.
1. From previous section, we know that top 3 brand that customer buy without getting other products are La Moderna, HSE and Sobrab Bkstrp.
2. We also know that La Moderna sells only pasta and HSE/Sobrab Bkstrp sell only syrups.
3. Since we know that 41.78% of customers who buy pasta will also buy pasta sauce and 16.34% of customers who buy syrups will also buy pancake mixes, we can include these brand a campaign like<br>
    -"You can buy products from La Moderna brand with any pasta sauce at discounted price"<br>
    -"You can buy products from HSE/Sobrab Bkstrp brand with any pancake mixes at discounted price"<br>